In [1]:
# perform Decision tree classifier on iris data
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier

iris = load_iris()
X = iris.data[:, 2:] # pedal length and width
y = iris.target

tree_clf = DecisionTreeClassifier(max_depth=2)
tree_clf.fit(X, y)
tree_clf.predict([[5.1, 1.7]])


array([1])

In [2]:
# visualize the decision tree we made above 
from sklearn.tree import export_graphviz

export_graphviz(
    tree_clf,
    out_file=image_path("iris_tree.dot"),
    feature_names=iris.feature_names[2:],
    class_names=iris.target_names,
    rounded=True,
    filled=True
)


NameError: name 'image_path' is not defined

In [6]:
# Decision tree can estimate class probability
tree_clf.predict([[5, 1.5]])
tree_clf.predict_proba([[5, 1.5]])


array([[ 0.        ,  0.90740741,  0.09259259]])

In [7]:
tree_clf.predict_proba([[6, 1.5]])

array([[ 0.        ,  0.90740741,  0.09259259]])

In [8]:
# perform Decision Tree Regression on iris data
from sklearn.tree import DecisionTreeRegressor

tree_reg = DecisionTreeRegressor(max_depth=2)
tree_reg.fit(X, y)
tree_reg.predict([[5, 1.5]])

array([ 1.09259259])

In [11]:
# exercise7, train a Decision Tree on moons dataset
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

X, y = make_moons(n_samples=10000, noise=0.4, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# find good hyperparameters for Decision Tree Classifier
param_grid = {'max_leaf_nodes': list(range(2, 100)), 'min_samples_split': [2, 3, 4]}
tree_clf = DecisionTreeClassifier(random_state=42)
grid_search = GridSearchCV(tree_clf, param_grid, n_jobs=-1, verbose=1)
grid_search.fit(X_train, y_train)





Fitting 3 folds for each of 294 candidates, totalling 882 fits


[Parallel(n_jobs=-1)]: Done 592 tasks      | elapsed:    1.7s
[Parallel(n_jobs=-1)]: Done 882 out of 882 | elapsed:    2.1s finished


GridSearchCV(cv=None, error_score='raise',
       estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=42,
            splitter='best'),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'min_samples_split': [2, 3, 4], 'max_leaf_nodes': [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, 

In [12]:
grid_search.best_estimator_

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=17,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=42,
            splitter='best')

In [16]:
# since grid search already trained the model using the best parameters, so we can directly use this model for prediction
y_pred = grid_search.predict(X_test)
print(accuracy_score(y_test, y_pred))


0.8695


In [19]:
# exercise 8
from sklearn.model_selection import ShuffleSplit
from sklearn.base import clone
import numpy as np

# generate 1000 subsets of the training data, each contains 100 instances
n_subsets = 1000
n_instances = 100
mini_sets = []

split = ShuffleSplit(n_splits=n_subsets, test_size = len(X_train)-n_instances, random_state=42)
for (mini_train_index, mini_test_index) in split.split(X_train):
    X_mini_train = X_train[mini_train_index]
    y_mini_train = y_train[mini_train_index]
    mini_sets.append((X_mini_train, y_mini_train))
    
# train Decision tree on each of the 1000 subsets
forest = [clone(grid_search.best_estimator_) for _ in range(n_subsets)]
accuracy_scores = []

for tree, (X_mini_train, y_mini_train) in zip(forest, mini_sets):
    tree.fit(X_mini_train, y_mini_train)
    y_pred = tree.predict(X_test)
    accuracy_scores.append(accuracy_score(y_test,y_pred))
    
np.mean(accuracy_scores)

0.80544949999999993

In [22]:
# generate predictions for the 1000 subsets and keep the most frequent
from scipy.stats import mode

Y_pred = np.empty([n_subsets, len(X_test)], dtype=np.unit8)
for tree_index, tree in enumerate(forest):
    Y_pred[tree_index] = tree.predict(X_test)
    
y_pred_majority_votes, n_votes = mode(Y_pred, axis=0)

AttributeError: 'module' object has no attribute 'unit8'

In [ ]:
accuracy_score(y_test, y_pred_majority_votes.reshape(-1))